In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 81.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/Book1.xlsx'
excel_data = pd.ExcelFile(file_path)

sheet_names = excel_data.sheet_names
print(sheet_names)

df = excel_data.parse(sheet_names[0])
print(df.head())


['Clean Data Del']
    SourceWH Order Date  Ship Date    Mode  Ext Transporter  Customer Zip  \
0  Bangalore 2024-04-09 2024-04-11  Surface       Delhivery        600053   
1  Bangalore 2024-04-25 2024-04-29  Surface       Delhivery        410201   
2  Bangalore 2024-05-10 2024-05-10  Surface      Xpressbees        624001   
3  Bangalore 2024-06-08 2024-06-10  Surface       Delhivery        505302   
4  Bangalore 2024-06-22 2024-06-24      Air       Delhivery        587311   

         State  Weight  Length  Breadth  Height     lbh First attempt Date  \
0   Tamil Nadu  5674.0    34.0     14.0    14.0  1.3328         2024-04-17   
1  Maharashtra  3354.4    22.0     11.0     9.0  0.4356         2024-05-02   
2   Tamil Nadu  3030.4    26.0     21.0    16.0  1.7472         2024-05-13   
3    Telangana  1330.0    40.0     32.0    14.0  3.5840         2024-06-13   
4    Karnataka   885.0    25.0     15.0    12.0  0.9000         2024-06-27   

  Delivery Date  No of Attempt FA TAT Map  PRICE 

In [ ]:
print(df.info())
print(df.describe())

df = df.dropna()
df['Weight'] = df['Weight'].astype(float)
df['PRICE'] = df['PRICE'].astype(float)

df['FA TAT'] = df['FA TAT'].str.replace('+', '').astype(float)  # Remove '+' and convert to int

df['lbh'] = df['lbh'].astype(str)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187548 entries, 0 to 187547
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   SourceWH            187548 non-null  object        
 1   Order Date          187548 non-null  datetime64[ns]
 2   Ship Date           187548 non-null  datetime64[ns]
 3   Mode                187548 non-null  object        
 4   Ext Transporter     187548 non-null  object        
 5   Customer Zip        187548 non-null  int64         
 6   State               187548 non-null  object        
 7   Weight              187548 non-null  float64       
 8   Length              187548 non-null  float64       
 9   Breadth             187548 non-null  float64       
 10  Height              187548 non-null  float64       
 11  lbh                 187548 non-null  float64       
 12  First attempt Date  187548 non-null  datetime64[ns]
 13  Delivery Date       187548 no

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# Define features and target variables
features = ['SourceWH', 'State', 'Ext Transporter', 'Weight', 'Length', 'Breadth', 'Height']
X = df[features]
y_price = df['PRICE']

# Convert 'FA TAT' to datetime objects if they are not already, handling errors
df['FA TAT'] = pd.to_datetime(df['FA TAT'], errors='coerce')

# Calculate minimum TAT for subtraction (after handling potential NaTs)
min_tat = df['FA TAT'].min()  # Find the minimum TAT value first
y_tat = df['FA TAT'].apply(lambda x: (x - min_tat).days if pd.notnull(x) else None)  # Handle NaTs

# Handle categorical features (One-Hot Encoding for 'State', 'SourceWH', and 'Ext Transporter')
ohe = OneHotEncoder(handle_unknown='ignore')  # 'ignore' to handle new categories in test data
X_encoded = ohe.fit_transform(X[['State', 'SourceWH', 'Ext Transporter']]) # Include 'Ext Transporter' in encoding

# Convert sparse matrix to DataFrame for easier handling
X_encoded_df = pd.DataFrame.sparse.from_spmatrix(X_encoded)
# Get feature names for one-hot encoded columns
X_encoded_df.columns = ohe.get_feature_names_out(['State', 'SourceWH', 'Ext Transporter']) # Update feature names

X_other = X.drop(['State', 'SourceWH', 'Ext Transporter'], axis=1) # Drop encoded columns, no need to reset index
X_final = pd.concat([X_other, X_encoded_df], axis=1)  # Combine encoded and other features

# Align y_price with X_final after handling NaNs in y_tat
y_price = y_price[y_tat.notna()] # Subset y_price to match rows in X_final

# Drop rows with NaN in y_tat *after* creating X_final to ensure alignment
X_final = X_final[y_tat.notna()]
y_tat = y_tat[y_tat.notna()]

# Split the data (using the encoded features)
X_train, X_test, y_train_price, y_test_price = train_test_split(X_final, y_price, test_size=0.2, random_state=42)
X_train, X_test, y_train_tat, y_test_tat = train_test_split(X_final, y_tat, test_size=0.2, random_state=42)

# Train the models
price_model = LinearRegression().fit(X_train, y_train_price)
tat_model = LinearRegression().fit(X_train, y_train_tat)

# Make predictions
price_predictions = price_model.predict(X_test)
tat_predictions = tat_model.predict(X_test)

# Create a new DataFrame for predictions to avoid length mismatch
predictions_df = pd.DataFrame({
    'price_predictions': price_predictions,
    'tat_predictions': tat_predictions
}, index=X_test.index)  # Use the index of X_test

# Optionally, you can merge these predictions back into your original DataFrame 'df'
df = df.merge(predictions_df, how='left', left_index=True, right_index=True)

print(price_predictions)
print(tat_predictions)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:785: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:785: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:785: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:785: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[45.82130374 35.64758559 29.5131939  ... 35.50547852 37.02898764
 44.1714536 ]
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Define a function to optimize and select the best courier
def optimize_courier(df, SourceWH, state):

    # Filter the data for the given warehouse and state
    filtered_df = df[(df['SourceWH'] == SourceWH) & (df['State'] == state)]

    # Handle potential NaN values in predictions
    filtered_df.fillna({'price_predictions': 0, 'tat_predictions': 0}, inplace=True)

    # Initialize the optimization problem
    model = LpProblem(name="courier-selection", sense=LpMinimize)

    # Create decision variables for each courier
    couriers = filtered_df['Ext Transporter'].unique()
    dec_vars = LpVariable.dicts("Ext Transporter", couriers, lowBound=0, cat='Binary')

    # Define the objective function: Minimize the sum of predicted price and TAT
    objective = lpSum([
        (filtered_df.loc[filtered_df['Ext Transporter'] == courier, 'price_predictions'].values[0] +
         filtered_df.loc[filtered_df['Ext Transporter'] == courier, 'tat_predictions'].values[0]) * dec_vars[courier]
        for courier in couriers
    ])
    model += objective

    # Add constraints: ensure only one courier is selected
    model += lpSum([dec_vars[courier] for courier in couriers]) == 1

    # Solve the problem
    status = model.solve() # Capture the status of the solver

    if status == 1: # 1 indicates an optimal solution was found
        best_courier = [v.name.replace("Ext Transporter_", "") for v in model.variables() if v.varValue is not None and v.varValue > 0]
        if best_courier:
            best_courier = best_courier[0]
        else:
            best_courier = "No feasible solution for any courier"
    else:
        best_courier = "No solution found" # Handle cases where no solution is found

    return best_courier

In [ ]:
def get_best_courier(SourceWH, state):
    best_courier = optimize_courier(df, SourceWH, state)
    return best_courier

# Example usage
warehouse = 'Delhi/Gurgaon'  # Replace with actual warehouse
state = 'Madhya Pradesh'          # Replace with actual state

best_courier = get_best_courier(warehouse, state)
print(f"The best courier for {warehouse} to {state} is: {best_courier}")


The best courier for Delhi/Gurgaon to Madhya Pradesh is: Ext_Transporter_Xpressbees


<ipython-input-6-37bc9adf1ed9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.fillna({'price_predictions': 0, 'tat_predictions': 0}, inplace=True)
